In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
path_in = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/raw/"
path_out = "/Users/sofia/Documents/Repos/skytruth-30x30/data/data/processed/"

### Create locations table

In [3]:
eez = gpd.read_file(path_out + "/administrative/eez_area_mollweide.shp")
regions = gpd.read_file(path_out + "/administrative/eez_regions.shp")
hs = gpd.read_file(path_in + "/high_seas/high_seas.shp")

In [ ]:
def add_location_iso(df):
    # Create new column "iso" that has the field "ISO_SOV1" for all rows except those in which ISO_SOV2 and ISO_SOV3 are not null. In such cases concatenate ISO_SOV1, ISO_SOV2 and ISO_SOV3
    return df.assign(iso =lambda row: ';'.join(filter(None, (row[["ISO_SOV1", 'ISO_SOV2', 'ISO_SOV3']]))))

def add_region_iso(df):
    return df.assign(region = lambda row: row['iso'].str.split(' ').str[0])

def expand_multiple_locations(df):
    mask = df['iso'].str.contains(';', na=False)
    split_rows = df[mask].copy()
    split_rows['iso'] = split_rows['iso'].str.split(';')
    split_rows = split_rows.explode('iso')
    return pd.concat([df[~mask], split_rows], ignore_index=True)

def calculate_area(df):
    glob = {'location_id':'GLOB', 'total_marine_area':361000000, 'location_type':'worldwide'}
    marine_areas = df.groupby(['iso']).agg({'AREA_KM2': 'sum'}).reset_index().assign(location_type = 'country')
    regions_areas = df.groupby(['region']).agg({'AREA_KM2': 'sum'}).reset_index().rename(columns={'region': 'iso'}).assign(location_type = 'region')
    
    return pd.concat([marine_areas, regions_areas], ignore_index=True).append(glob, ignore_index=True)
    

def add_location_name(df):
    return df.assign(name_iso = lambda row: iso_country_mapping.get(row.iso, None))



In [4]:
# Create new column "iso" that has the field "ISO_SOV1" for all rows except those in which ISO_SOV2 and ISO_SOV3 are not null. In such cases concatenate ISO_SOV1, ISO_SOV2 and ISO_SOV3
eez['iso'] = eez['ISO_SOV1']
eez.loc[eez['ISO_SOV2'].notnull(), 'iso'] = eez['ISO_SOV1'] + ";" + eez['ISO_SOV2']
eez.loc[eez['ISO_SOV3'].notnull(), 'iso'] = eez['ISO_SOV1'] + ";" + eez['ISO_SOV2'] + ";" + eez['ISO_SOV3']

In [5]:
# Create a mask for rows with multiple values in 'iso_code'
mask = eez['iso'].str.contains(';', na=False)

# Split the 'iso_code' values and create separate rows only for rows with multiple values
split_rows = eez[mask].copy()
split_rows['iso'] = split_rows['iso'].str.split(';')
split_rows = split_rows.explode('iso')

# Keep rows with single values in 'iso_code'
single_value_rows = eez[~mask]

# Concatenate the exploded rows with the single value rows
eez_new = pd.concat([single_value_rows, split_rows], ignore_index=True)

eez_new.shape

(337, 33)

In [ ]:
iso_country_mapping = {
    'USA': 'United States',
    'GBR': 'United Kingdom',
    'NZL': 'New Zealand',
    'FRA': 'France',
    'WSM': 'Samoa',
    'TON': 'Tonga',
    'CHL': 'Chile',
    'URY': 'Uruguay',
    'PER': 'Peru',
    'BRA': 'Brazil',
    'KIR': 'Kiribati',
    'ARG': 'Argentina',
    'AUS': 'Australia',
    'COM': 'Comoros',
    'MDG': 'Madagascar',
    'ZAF': 'South Africa',
    'MUS': 'Mauritius',
    'VUT': 'Vanuatu',
    'NAM': 'Namibia',
    'TLS': 'Timor-Leste',
    'COG': 'Republic of the Congo',
    'AGO': 'Angola',
    'MOZ': 'Mozambique',
    'KEN': 'Kenya',
    'PNG': 'Papua New Guinea',
    'TZA': 'Tanzania',
    'SLB': 'Solomon Islands',
    'SYC': 'Seychelles',
    'COD': 'Democratic Republic of the Congo',
    'ATG': 'Antigua and Barbuda',
    'NLD': 'Netherlands',
    'PRT': 'Portugal',
    'BHS': 'The Bahamas',
    'BRB': 'Barbados',
    'MEX': 'Mexico',
    'CPV': 'Cape Verde',
    'ESP': 'Spain',
    'PAN': 'Panama',
    'CRI': 'Costa Rica',
    'DMA': 'Dominica',
    'DOM': 'Dominican Republic',
    'GTM': 'Guatemala',
    'DNK': 'Denmark',
    'GMB': 'Gambia',
    'GIB': 'Gibraltar',
    'GRD': 'Grenada',
    'SLE': 'Sierra Leone',
    'ISL': 'Iceland',
    'JAM': 'Jamaica',
    'MRT': 'Mauritania',
    'HTI': 'Haiti',
    'KNA': 'Saint Kitts and Nevis',
    'LCA': 'Saint Lucia',
    'VCT': 'Saint Vincent and the Grenadines',
    'TTO': 'Trinidad and Tobago',
    'SLV': 'El Salvador',
    'BLZ': 'Belize',
    'CUB': 'Cuba',
    'SEN': 'Senegal',
    'VEN': 'Venezuela',
    'CAN': 'Canada',
    'NIC': 'Nicaragua',
    'GUY': 'Guyana',
    'COL': 'Colombia',
    'IRL': 'Ireland',
    'GNB': 'Guinea-Bissau',
    'GIN': 'Guinea',
    'CIV': 'Ivory Coast',
    'LBR': 'Liberia',
    'HND': 'Honduras',
    'ECU': 'Ecuador',
    'ESH': 'Western Sahara',
    'SUR': 'Suriname',
    'MAR': 'Morocco',
    'ARE': 'United Arab Emirates',
    'CYP': 'Cyprus',
    'ERI': 'Eritrea',
    'EGY': 'Egypt',
    'GEO': 'Georgia',
    'IRN': 'Iran',
    'LBN': 'Lebanon',
    'LBY': 'Libya',
    'MLT': 'Malta',
    'OMN': 'Oman',
    'SAU': 'Saudi Arabia',
    'LKA': 'Sri Lanka',
    'SDN': 'Sudan',
    'SYR': 'Syria',
    'TGO': 'Togo',
    'GRC': 'Greece',
    'TUR': 'Turkey',
    'MCO': 'Monaco',
    'TUN': 'Tunisia',
    'MNE': 'Montenegro',
    'ALB': 'Albania',
    'BGR': 'Bulgaria',
    'PSE': 'Palestine',
    'KWT': 'Kuwait',
    'IRQ': 'Iraq',
    'BHR': 'Bahrain',
    'QAT': 'Qatar',
    'YEM': 'Yemen',
    'ISR': 'Israel',
    'JOR': 'Jordan',
    'DJI': 'Djibouti',
    'BGD': 'Bangladesh',
    'NGA': 'Nigeria',
    'CMR': 'Cameroon',
    'STP': 'São Tomé and Príncipe',
    'BIH': 'Bosnia and Herzegovina',
    'MHL': 'Marshall Islands',
    'PLW': 'Palau',
    'PHL': 'Philippines',
    'TWN': 'Taiwan',
    'SGP': 'Singapore',
    'THA': 'Thailand',
    'VNM': 'Vietnam',
    'KOR': 'South Korea',
    'BRN': 'Brunei',
    'PRK': 'North Korea',
    'KHM': 'Cambodia',
    'CHN': 'China',
    'EST': 'Estonia',
    'FIN': 'Finland',
    'SWE': 'Sweden',
    'LTU': 'Lithuania',
    'NOR': 'Norway',
    'BEL': 'Belgium',
    'DEU': 'Germany',
    'LVA': 'Latvia',
    'HRV': 'Croatia',
    'ITA': 'Italy',
    'UKR': 'Ukraine',
    'ROU': 'Romania',
    'JPN': 'Japan',
    'IND': 'India',
    'PAK': 'Pakistan',
    'TKM': 'Turkmenistan',
    'AZE': 'Azerbaijan',
    'KAZ': 'Kazakhstan',
    'MMR': 'Myanmar',
    'POL': 'Poland',
    'BEN': 'Benin',
    'SVN': 'Slovenia',
    'MYS': 'Malaysia',
    'ATA': 'Antarctica',
    'TUV': 'Tuvalu',
    'FJI': 'Fiji',
    'FSM': 'Micronesia',
    'GNQ': 'Equatorial Guinea',
    'MDV': 'Maldives',
    'SOM': 'Somalia',
    'NRU': 'Nauru',
    'GAB': 'Gabon',
    'IDN': 'Indonesia',
    'DZA': 'Algeria',
    'GHA': 'Ghana',
    'RUS': 'Russia'
}

In [7]:
def get_name(country):
    return iso_country_mapping.get(country, None)

# Apply the function to create the 'PARENT_ISO' column
eez_new['name_iso'] = eez_new['iso'].apply(get_name)

In [9]:
# List of dictionaries for data in Region_ISO3_PP.txt (list of regions used in the Protected Planet database)
regions_data = [
    {
        'region_iso': 'AS',
        'region_name': 'Asia & Pacific',
        'country_iso_3s': [
            "AFG", "ASM", "AUS", "BGD", "BRN", "BTN", "CCK", "CHN", "COK", "CXR", "FJI", "FSM", "GUM", "HKG", "IDN",
            "IND", "IOT", "IRN", "JPN", "KHM", "KIR", "KOR", "LAO", "LKA", "MAC", "MDV", "MHL", "MMR", "MNG", "MNP",
            "MYS", "NCL", "NFK", "NIU", "NPL", "NRU", "NZL", "PAK", "PCN", "PHL", "PLW", "PNG", "PRK", "PYF", "SGP",
            "SLB", "THA", "TKL", "TLS", "TON", "TUV", "TWN", "VNM", "VUT", "WLF", "WSM"
        ]
    },
    {
        'region_iso': 'AF',
        'region_name': 'Africa',
        'country_iso_3s': [
            "AGO", "BDI", "BEN", "BFA", "BWA", "CAF", "CIV", "CMR", "COD", "COG", "COM", "CPV", "DJI", "DZA", "EGY",
            "ERI", "ESH", "ETH", "GAB", "GHA", "GIN", "GMB", "GNB", "GNQ", "KEN", "LBR", "LBY", "LSO", "MAR", "MDG",
            "MLI", "MOZ", "MRT", "MUS", "MWI", "MYT", "NAM", "NER", "NGA", "REU", "RWA", "SDN", "SEN", "SHN", "SLE",
            "SOM", "SSD", "STP", "SWZ", "SYC", "TCD", "TGO", "TUN", "TZA", "UGA", "ZAF", "ZMB", "ZWE"
        ]
    },
    {
        'region_iso': 'EU',
        'region_name': 'Europe',
        'country_iso_3s': [
            "ALA", "ALB", "AND", "ARM", "AUT", "AZE", "BEL", "BGR", "BIH", "BLR", "CHE", "CYP", "CZE", "DEU", "DNK",
            "ESP", "EST", "FIN", "FRA", "FRO", "GBR", "GEO", "GGY", "GIB", "GRC", "HRV", "HUN", "IMN", "IRL", "ISL",
            "ISR", "ITA", "JEY", "KAZ", "KGZ", "LIE", "LTU", "LUX", "LVA", "MCO", "MDA", "MKD", "MLT", "MNE", "NLD",
            "NOR", "POL", "PRT", "ROU", "RUS", "SJM", "SMR", "SRB", "SVK", "SVN", "SWE", "TJK", "TKM", "TUR", "UKR",
            "UZB", "VAT"
        ]
    },
    {
        'region_iso': 'SA',
        'region_name': 'Latin America & Caribbean',
        'country_iso_3s': [
            "ABW", "AIA", "ARG", "ATG", "BES", "BHS", "BLM", "BLZ", "BMU", "BOL", "BRA", "BRB", "CHL", "COL", "CRI",
            "CUB", "CUW", "CYM", "DMA", "DOM", "ECU", "FLK", "GLP", "GRD", "GTM", "GUF", "GUY", "HND", "HTI", "JAM",
            "KNA", "LCA", "MAF", "MEX", "MSR", "MTQ", "NIC", "PAN", "PER", "PRI", "PRY", "SLV", "SUR", "SXM", "TCA",
            "TTO", "UMI", "URY", "VCT", "VEN", "VGB", "VIR"
        ]
    },
    {
        'region_iso': 'PO',
        'region_name': 'Polar',
        'country_iso_3s': [
            "ATF", "BVT", "GRL", "HMD", "SGS"
        ]
    },
    {
        'region_iso': 'NA',
        'region_name': 'North America',
        'country_iso_3s': [
            "CAN", "SPM", "USA"
        ]
    },
    {
        'region_iso': 'WA',
        'region_name': 'West Asia',
        'country_iso_3s': [
            "ARE", "BHR", "IRQ", "JOR", "KWT", "LBN", "OMN", "PSE", "QAT", "SAU", "SYR", "YEM"
        ]
    },
    {
        'region_iso': 'AT', # this region is not in the Protected Planet database
        'region_name': 'Antartica',
        'country_iso_3s': [
            "ATA"
        ]
    }
]

# Convert the region data to a dictionary that maps each country to its region name
country_to_region = {}
name_to_region = {}
for region in regions_data:
    for country in region['country_iso_3s']:
        country_to_region[country] = region['region_iso']
        name_to_region[country] = region['region_name']

In [10]:
eez_new['region'] = eez_new['iso'].map(country_to_region)
eez_new['region_name'] = eez_new['iso'].map(name_to_region)

In [11]:
marine_areas = eez_new.groupby(['iso', 'name_iso']).agg({'AREA_KM2': 'sum'}).reset_index()
marine_areas = marine_areas.rename(columns={'iso': 'location_id', 'name_iso':'location_name', 'AREA_KM2': 'total_marine_area'})
marine_areas['location_type'] = 'country'

In [12]:
regions_areas = eez_new.groupby(['region', 'region_name']).agg({'AREA_KM2': 'sum'}).reset_index()
regions_areas = regions_areas.rename(columns={'region': 'location_id', 'region_name':'location_name', 'AREA_KM2': 'total_marine_area'})
regions_areas['location_type'] = 'region'

In [13]:
global_area = pd.DataFrame({'location_id': ['GLOB'], 'location_name': ['Worldwide'], 'total_marine_area': [361000000], 'location_type': ['worldwide']})
hs_area = pd.DataFrame({'location_id': ['ABNJ'], 'location_name': ['High Seas'], 'total_marine_area': [hs['area_km2'].values[0]], 'location_type': ['country']})

In [14]:
# concat gl_df and hs_df to marine_areas
marine_areas2 = pd.concat([marine_areas, regions_areas, global_area, hs_area], ignore_index=True)


In [16]:
# Save the table as csv
marine_areas2.to_csv(path_out + "/tables/locations.csv", index=False)

### Create region_locations table

In [17]:
regions_df = pd.DataFrame([{'region_id': data['region_iso'], 'location_id': iso} for data in regions_data for iso in data['country_iso_3s']])
regions_df

,region_id,location_id
0,AS,AFG
1,AS,ASM
2,AS,AUS
3,AS,BGD
4,AS,BRN
...,...,...
244,WA,QAT
245,WA,SAU
246,WA,SYR
247,WA,YEM


In [18]:
regions_df.to_csv(path_out + '/tables/region_locations.csv', index=False)